<a href="https://colab.research.google.com/github/DanielDialektico/rag_agentes_langchain_curso/blob/main/notebooks/ragas_eval_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://dialektico.com/wp-content/uploads/2023/03/MiniLogoW4.png" alt="Dialéktico Logo" />

Este pequeño tutorial pertenece al curso de RAG y agentes con LangChain al que puedes acceder mediante la siguiente URL: https://www.youtube.com/playlist?list=PLlWTv9_GeWd32stuEMWpYOnxiVxnXaU6q

Sigue los videos del curso para recibir instrucciones y contexto sobre la ejecución de este Notebook.

<br>

# Se instalan e importan las librerías

In [ ]:
# Se instalan las librerías.
!pip install langchain==0.3.21
!pip install langchain-community==0.3.20
!pip install beautifulsoup4==4.13.3
!pip install tiktoken==0.9.0
!pip install pypdf==5.4.0
!pip install langchain-huggingface==0.1.2
!pip install langchain_deepseek==0.1.2
!pip install ragas==0.2.15

In [ ]:
# Se importan las librerías.
import os
import warnings
import sys
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_deepseek import ChatDeepSeek
import bs4
from ragas import EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, SemanticSimilarity, FactualCorrectness
from ragas.embeddings import LangchainEmbeddingsWrapper
from google.colab import userdata
from uuid import uuid4
from IPython.display import display, HTML
from pydantic import BaseModel, Field
from typing import List


warnings.filterwarnings('ignore')

<br>

# Se declaran los modelos a utilizar

In [ ]:
# Se añade la API key como variable de ambiente desde un secreto en Colab.
os.environ["DEEPSEEK_API_KEY"] = userdata.get('DEEPSEEK_API_KEY')

model = ChatDeepSeek(
      model="deepseek-chat",
      temperature=0,
      max_tokens=1000
      )

In [ ]:
# Se prepara el algoritmo de embeddings a utilizar.
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

<br>

# Se indexan documentos vectorizados de forma local

In [ ]:
# Se carga la información desde distintas fuentes
web_loader = WebBaseLoader(web_paths=["https://dialektico.com/cama-ultra-lujosa-para-gatos-dialektiroyal-comfort/"])
documents = []

for doc in os.listdir('documents'):
  pdf_loader = PyPDFLoader('documents/'+ doc)
  async for page in pdf_loader.alazy_load():
      documents.append(page)

async for doc in web_loader.alazy_load():
    documents.append(doc)

# Se crea vector store en memoria para almacenar vectores.
vector_store = InMemoryVectorStore(embedding=embeddings_model)

# Se instancia el CharacterTextSplitter.
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=1000, chunk_overlap=200
)

# Se divide el texto de los documentos utilizando el tokenizador tiktoken.
chunks = text_splitter.split_documents(documents)

# Se almacenan los vectores.
vector_store.add_documents(documents=chunks, ids=[str(uuid4()) for _ in range(len(chunks))])

<br>

# Se crear el flujo de respuestas con RAG

In [ ]:
def rag_answer(user_prompt: str):
  # Se crea template para RAG.
  prompt_template = """Utiliza los siguientes fragmentos de contexto para responder la
  pregunta al final. Si no sabes la respuesta, simplemente di que no la sabes, no
  intentes inventar una respuesta.
  Utiliza pocas oraciones y mantén la respuesta lo más concisa posible.

  CONTEXTO: {context}

  PREGUNTA: {question}

  Respuesta útil:"""

  # Se recuperan documentos similares.
  retrieved_docs = vector_store.similarity_search(user_prompt, k=5)
  context = "\n\n".join(doc.page_content for doc in retrieved_docs)

  # Se añade el contexto al prompt.
  prompt = PromptTemplate.from_template(prompt_template)
  messages = [
    SystemMessage("""Eres un asistente virtual de una tienda en línea llamada
    dialektipet. Responde a las preguntas del usuario con la información
    proporcionada"""),
    HumanMessage(prompt.invoke({"question": user_prompt, "context": context}).text),
]


  # Se genera la respuesta.
  response = model.invoke(messages).content

  return (response, context)

In [ ]:
# Se prueba la función.
response, context = rag_answer('¿Cómo protegen mi información al realizar un pago?')

In [ ]:
response

In [ ]:
context

<br>

# Evaluando el RAG con Ragas

Para la evaluación de las respuestas del modelo, utilizaremos la librería ragas, la cual nos permite realizar mediciones mediante diferentes métricas.

Más información en:


*   https://docs.ragas.io/en/stable/
*   https://arxiv.org/pdf/2309.15217



## Se crea el conjunto de datos para su evaluación

In [ ]:
# Preguntas del usuari.
sample_queries = [
    "¿Cómo protegen mi información al realizar un pago?",
    "¿Cuál es el precio y las tallas de la playera de algodón para perro?",
    "¿Qué es Dialektipet?",
    "¿Qué hago si aún no recibo mi reembolso?",
    "¿Se aceptan devoluciones de prouductos usados?"
]

# Respuestas esperadas.
expected_responses = [
    "Nuestras plataformas de Pagos en línea manejan un candado SSL y otros sistemas de seguridad, y sobre todo se tratan de empresas reconocidas con mucha experiencia y excelente reputación en el manejo de información de pagos de cliente en México y en el mundo.",
    "Las tallas son XS, S, M, L, y el precio es de  $14.99 USD.",
    "DialektiPet es una tienda en línea especializada en ofrecer los mejores accesorios y productos para mascotas.",
    "Si aún no recibiste un reembolso, primero revisa de nuevo tu cuenta bancaria. Luego, comunícate con la empresa de tu tarjeta de crédito. [...] puedes contactarnos escribiendo a contacto@dialektipet.com",
    "No se aceptarán devoluciones de productos usados, abiertos o dañados por el cliente."
]


In [ ]:
# Se añaden las respuestas del modelo y el contexto obtenido mediante el RAG.
dataset = []

for query, reference in zip(sample_queries, expected_responses):
    response, context = rag_answer(query)
    dataset.append({
        "user_input": query,
        "retrieved_contexts": [context],
        "response": response,
        "reference": reference,
    })

evaluation_dataset = EvaluationDataset.from_list(dataset)

In [ ]:
dataset[0]

<br>

## Evaluación con métricas de Ragas

Para esta evaluación utilizaremos medidas del tipo "LLM como juez", donde se utiliza un modelo para determinar la puntuación de una respuesta y/o contexto.

Se utilizarán las siguientes tres:


*   LLM Context Recall
*   Faithfulness
*   Factual Correctness

Donde:

1. **LLM Context Recall**

Evalúa qué tanto del contenido de la respuesta de referencia puede encontrarse en los documentos recuperados.

Imagina que la respuesta ideal (la referencia) tiene 4 afirmaciones o datos.

Si 3 de esas afirmaciones están respaldadas por los documentos recuperados, el context recall es:

$\text{Context Recall} = \frac{3}{4} = 0.75$

Útil para saber si el sistema recuperó el contenido necesario para construir una buena respuesta.

2. **Faithfulness**

Mide si la respuesta generada realmente se basa en los documentos recuperados.

Si la respuesta generada tiene 5 afirmaciones y 4 de ellas están en los documentos recuperados, la puntuación es:

$\text{Faithfulness} = \frac{4}{5} = 0.8$

Ayuda a verificar que el modelo no esté "inventando" cosas fuera del contexto recuperado.


3. **Factual Correctness**

Mide si la respuesta generada coincide con la respuesta correcta (referencia). Aquí se consideran:

TP (True Positive): afirmaciones correctas que sí están en la referencia.

FP (False Positive): afirmaciones incorrectas (no están en la referencia).

FN (False Negative): afirmaciones que faltaron (sí están en la referencia, pero no en la respuesta).

4. **Semantic Similarity**

Se refiere a la evaluación del parecido semántico entre la respuesta generada y la respuesta verdadera (ground truth). Esta evaluación se basa en comparar la respuesta generada con la referencia, y los valores obtenidos oscilan entre 0 y 1. Un puntaje más alto indica una mejor alineación semántica entre ambas respuestas.
Esta evaluación utiliza un modelo de embeddings para calcular el puntaje de similitud semántica.

Más información en: https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/

Creamos un conjunto de datos de prueba:

In [ ]:
# Se crea conjunto de datos de prueba.
sample = {
    "user_input": "¿Dónde está la torre Eiffel?",
    "retrieved_contexts": ["La torre Eiffel está en París"],
    "response": "La torre Eiffel está en México",
    "reference": "La torre Eiffel está localizada en París."
}

test_dataset_list = [sample]

test_dataset_false = EvaluationDataset.from_list(test_dataset_list)

In [ ]:
# Se crea conjunto de datos de prueba.
sample = {
    "user_input": "¿Dónde está la torre Eiffel?",
    "retrieved_contexts": ["La torre Eiffel está en París"],
    "response": "La torre Eiffel está en París",
    "reference": "La torre Eiffel está localizada en París."
}

test_dataset_list = [sample]

test_dataset_true = EvaluationDataset.from_list(test_dataset_list)

In [ ]:
# Se declara el modelo a utilizar para la evaluación.
evaluator_llm = LangchainLLMWrapper(langchain_llm=model)

Realizamos pruebas con diferentes métricas:

### LLM como juez

In [ ]:
# Se ejecuta la evaluación.
result = evaluate(
    dataset=test_dataset_false,
    metrics=[Faithfulness()],
    llm=evaluator_llm,
)

print(result)

In [ ]:
# Se ejecuta la evaluación.
result = evaluate(
    dataset=test_dataset_true,
    metrics=[Faithfulness()],
    llm=evaluator_llm,
)

print(result)

### Diferenciación matemática

In [ ]:
# Se añade el modelo de emebeddings.
embeddings=LangchainEmbeddingsWrapper(embeddings_model)

In [ ]:
# Se ejecuta la evaluación.
result = evaluate(
    dataset=test_dataset_false,
    metrics=[SemanticSimilarity()],
    embeddings=embeddings,
)

print(result)

In [ ]:
# Se ejecuta la evaluación.
result = evaluate(
    dataset=test_dataset_true,
    metrics=[SemanticSimilarity()],
    embeddings=embeddings,
)

print(result)

## Evaluación del conjunto de datos creado

In [ ]:
# Se ejecuta la evaluación.
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), SemanticSimilarity()],
    llm=evaluator_llm,
    embeddings=embeddings
)

In [ ]:
print(result)

Más información en:


*   https://docs.ragas.io/en/stable/howtos/integrations/langchain/#building-a-simple-qa-application
*   https://docs.ragas.io/en/latest/references/evaluate/#ragas.evaluation.evaluate



<br>

In [ ]:
# Dialektico Machine learning practices © 2025 by Daniel Antonio García Escobar
# is licensed under CC BY-NC 4.0. To view a copy of this license,
# visit https://creativecommons.org/licenses/by-nc/4.0/

# Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International
# Public License